# Nibiru PERP (Perpetuals) Queries and Transactions
Perpetuals is an instrument for margin trading.
See documentation here: https://nibiru.fi/docs/ecosystem/nibi-perps/

≈≈### 1. Install Nibiru package

In [ ]:
!pip install nibiru

### 2. Create chain client and run queries
Using testnet-3 (aka itn-3) endpoint run queiries

In [ ]:
import json
from nibiru import Network, ChainClient, Msg, Coin, TxConfig
from nibiru.exceptions import QueryError

client = ChainClient(network=Network.testnet(3))

In [ ]:
# Shortcut to print responses
def print_json(obj):
    print(json.dumps(obj, indent=2))

# Pretty print trader positions
def print_trader_positions(trader_address):
    print(f"Trader {trader_address} positions:")
    print_json(
        client.query.perp.all_positions(trader_address)
    )

In [ ]:
# Query perp markets
markets = client.query.perp.markets()
print_json(markets)

In [ ]:
# Query perp positions of a trader
trader_address = "nibi1jle8khj3aennq24zx6g93aam9rt0fqhgyp4h52"
print_trader_positions(trader_address)

In [ ]:
# Query perp position of a trader on a specific market
pair = "ubtc:unusd"
try:
    position = client.query.perp.position(pair, trader_address)
    print_json(position)
except QueryError:
    print("Position not found")

### 3. Authenticate to run transactions
Assuming you have your wallet mnemonic key (or private key) and some test tokens from the faucet.
See: https://app.nibiru.fi/faucet

In [ ]:
mnemonic = "put your mnemonic here..."

# Let's set high gas multiplier to avoid out of gas errors. Could be customized per tx.
tx_config = TxConfig(gas_multiplier=5)
client = ChainClient(Network.testnet(3), tx_config=tx_config)
client.authenticate(mnemonic=mnemonic)

# Balances check
balances = client.query.get_bank_balances(client.address)
print_json(balances["balances"])

### 4. Run PERP transactions
Open perp position with 5x leverage:

In [ ]:
pair = "ubtc:unusd" # u-prefix means micro (1 unusd = 1/1000_000 NUSD)

output = client.tx.execute_msgs(
    Msg.perp.open_position(
        pair=pair,
        is_long=True,
        margin=10,
        leverage=5,
    ),
    wait_for_tx_resp=True,  # wait for block and get tx response
)
# Expected tx response code 0
print(f"tx response Code: {output['code']}")
if output["code"] == 0:
    print_trader_positions(client.address)

Add some margin if position becomes unhealthy (low margin ratio):

In [ ]:
output = client.tx.execute_msgs(
    Msg.perp.add_margin(pair=pair, margin=Coin(1, "unusd")),
    wait_for_tx_resp=True,
)
# Expected tx response code 0
print(f"tx response code: {output['code']}")
if output["code"] == 0:
    print_trader_positions(client.address)

Remove margin if needed:

In [ ]:
output = client.tx.execute_msgs(
    Msg.perp.remove_margin(pair=pair, margin=Coin(1, "unusd")),
    wait_for_tx_resp=True,
)
# Expected tx response code 0
print(f"tx response code: {output['code']}")
if output["code"] == 0:
    print_trader_positions(client.address)

Close position:

In [ ]:
# Close position
output = client.tx.execute_msgs(
    Msg.perp.close_position(pair=pair),
    wait_for_tx_resp=True,
)
# Expected tx response code 0
print(f"tx response code: {output['code']}")
if output["code"] == 0:
    print_trader_positions(client.address)